# #Final


# Fit and Predict Credit Information


I am going to fit and predit credit information of a competition, "Home Credit Default Risk" on Kaggle.


https://www.kaggle.com/c/home-credit-default-risk


## [Task 1] Create a Pipeline

In [1]:
import pandas as pd
import lightgbm as lgbm
from sklearn.metrics import roc_auc_score
import numpy as np
import heapq
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

/Users/kazukiegusa/.pyenv/versions/anaconda3-5.3.0/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_9.4.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
# get the train and test dataset
train = pd.read_csv('"Home Credit Default Risk"_application_train.csv')
test = pd.read_csv('"Home Credit Default Risk"_application_test.csv')

# concatenate the train dataset with the test dataset
train_test = train.append(test)

# create dummies
new_train_test = pd.get_dummies(train_test)

#  compute the correlation coefficients
corr_mat = new_train_test.corr()

# split the concatenated dataset into the train dataset and the test dataset
train_input = new_train_test[0: train.shape[0]].copy()
test_input = new_train_test[train.shape[0]:].copy()

/Users/kazukiegusa/.pyenv/versions/anaconda3-5.3.0/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [3]:
# LightGBM
X = train_input.drop("TARGET",axis=1).values
y = train_input.loc[:,'TARGET'].values

lgbm_train = lgbm.Dataset(X,y)

lgbm_eval = lgbm.Dataset(X, y, reference=lgbm_train)

lgbm_params = {'objective':'binary',
               'boosting_type': 'rf',
               'subsample': 0.623,
               'colsample_bytree': 0.7,
               'num_leaves': 127,
               'max_depth': 8,
               'seed': 99,
               'bagging_freq': 1,
               'n_jobs': 4}

model = lgbm.train(lgbm_params, lgbm_train, valid_sets=lgbm_eval)

[1]	valid_0's binary_logloss: 0.262326
[2]	valid_0's binary_logloss: 0.25312
[3]	valid_0's binary_logloss: 0.251118
[4]	valid_0's binary_logloss: 0.250335
[5]	valid_0's binary_logloss: 0.250268
[6]	valid_0's binary_logloss: 0.250072
[7]	valid_0's binary_logloss: 0.249893
[8]	valid_0's binary_logloss: 0.249758
[9]	valid_0's binary_logloss: 0.249673
[10]	valid_0's binary_logloss: 0.249562
[11]	valid_0's binary_logloss: 0.249642
[12]	valid_0's binary_logloss: 0.24955
[13]	valid_0's binary_logloss: 0.249641
[14]	valid_0's binary_logloss: 0.249725
[15]	valid_0's binary_logloss: 0.249621
[16]	valid_0's binary_logloss: 0.249718
[17]	valid_0's binary_logloss: 0.249586
[18]	valid_0's binary_logloss: 0.24948
[19]	valid_0's binary_logloss: 0.249417
[20]	valid_0's binary_logloss: 0.249535
[21]	valid_0's binary_logloss: 0.249576
[22]	valid_0's binary_logloss: 0.249498
[23]	valid_0's binary_logloss: 0.249422
[24]	valid_0's binary_logloss: 0.24936
[25]	valid_0's binary_logloss: 0.24932
[26]	valid_0's

In [4]:
# predict the test dataset
y_pred = model.predict(X)

# compute AUC
roc_auc_score(y,y_pred)

0.7570752634565502

In [5]:
# compute feature importances

train_features = [f for f in train_input if f not in ['SK_ID_CURR','TARGET']]

fi_lgbm = pd.DataFrame()

fi_lgbm['importance_split'] = model.feature_importance(importance_type='split')
fi_lgbm['importance_gain'] = model.feature_importance(importance_type='gain')

feat_labels = train_input[train_features].columns[0:]

indices_s = np.argsort(fi_lgbm['importance_split'])[::-1]
indices_g = np.argsort(fi_lgbm['importance_gain'])[::-1]

for f in range(train_input[train_features].shape[1]):
    print('%2d) %-*s %f' % (f+1, 30, feat_labels[indices_s[f]], fi_lgbm['importance_split'][indices_s[f]]))
    
for f in range(train_input[train_features].shape[1]):
    print('%2d) %-*s %f' % (f+1, feat_labels[indices_g[f]], fi_lgbm['importance_gain'][indices_g[f]]))    

 1) ORGANIZATION_TYPE_Insurance    0.000000
 2) ORGANIZATION_TYPE_Construction 0.000000
 3) ORGANIZATION_TYPE_Electricity  0.000000
 4) ORGANIZATION_TYPE_Emergency    0.000000
 5) ORGANIZATION_TYPE_Government   0.000000
 6) ORGANIZATION_TYPE_Housing      0.000000
 7) ORGANIZATION_TYPE_Industry: type 1 0.000000
 8) ORGANIZATION_TYPE_Industry: type 10 0.000000
 9) ORGANIZATION_TYPE_Industry: type 11 0.000000
10) ORGANIZATION_TYPE_Industry: type 13 0.000000
11) ORGANIZATION_TYPE_Industry: type 2 0.000000
12) ORGANIZATION_TYPE_Industry: type 3 0.000000
13) ORGANIZATION_TYPE_Industry: type 5 0.000000
14) ORGANIZATION_TYPE_Industry: type 6 0.000000
15) ORGANIZATION_TYPE_Industry: type 7 0.000000
16) ORGANIZATION_TYPE_Industry: type 8 0.000000
17) ORGANIZATION_TYPE_Industry: type 9 0.000000
18) ORGANIZATION_TYPE_Kindergarten 0.000000
19) ORGANIZATION_TYPE_Medicine     0.000000
20) ORGANIZATION_TYPE_Military     0.000000
21) ORGANIZATION_TYPE_Mobile       0.000000
22) ORGANIZATION_TYPE_Other  

IndexError: index 244 is out of bounds for axis 0 with size 244

In [ ]:
# draw a graph of the feature importances

ranking = np.argsort(-fi_lgbm['importance_split'])

f, ax = plt.subplots(figsize=(90, 90)) 

colors = ['red' if (x>=min(heapq.nlargest(10, fi_lgbm['importance_split'][ranking]))) else 'grey' \
          for x in fi_lgbm['importance_split'][ranking]]

sns.barplot(x=fi_lgbm['importance_split'][ranking], y=train_input[train_features].columns.values[ranking-1], orient='h', palette=colors)

ax.set_xlabel('feature importance',fontsize=70)
ax.set_yticklabels(feat_labels[indices_s-1],fontsize=30)
plt.tight_layout()
plt.show()

## [Task 2] Create a Definition of the AUC

In [9]:
# def of AUC

def compute_auc(train_test):
    # split the concatenated dataset into the train dataset and the test dataset
    train_input = train_test[0: train.shape[0]].copy()
    test_input = train_test[train.shape[0]:].copy()
    
    # LightGBM
    X = train_input.drop("TARGET",axis=1).values
    y = train_input.loc[:,'TARGET'].values
    
    lgbm_train = lgbm.Dataset(X,y)
    lgbm_eval = lgbm.Dataset(X, y, reference=lgbm_train)
    lgbm_params = {'objective':'binary',
               'boosting_type': 'rf',
               'subsample': 0.623,
               'colsample_bytree': 0.7,
               'num_leaves': 127,
               'max_depth': 8,
               'seed': 99,
               'bagging_freq': 1,
               'n_jobs': 4}
    
    model = lgbm.train(lgbm_params, lgbm_train, valid_sets=lgbm_eval)
    
    # predict the test dataset
    y_pred = model.predict(X)
    
    # compute AUC
    return roc_auc_score(y,y_pred)

## [Task 3] Preprocessing, Feature Enginnering and Fitting


### 1st Try

In [ ]:
# get top100 feature values strongly correlated with "TARGET"

top100_correlations = corr_mat.nlargest(101, "TARGET")["TARGET"].drop("TARGET")

top100_correlations_table = pd.DataFrame({
    'correlation': top100_correlations
})

top100_correlations_table

In [ ]:
# create a list of the 100 feature values

top100_list = list(top100_correlations.index)

top100_list

In [ ]:
# compute how many percentages each feature value includes missing values

total_of_top100 = new_train_test[top100_list].isnull().sum()
missing_ratio_of_top100 = total_of_top100 / len(new_train_test[top100_list])

missing_ratio_table_of_top100 = pd.DataFrame({
    'Total': total_of_top100,
    'missing_ratio': missing_ratio_of_top100
})

missing_ratio_table_of_top100

In [ ]:
# get top10 feature values whose missing ratio is in the above top100 list

top10_missing_ratio = missing_ratio_table_of_top100.nlargest(11, "missing_ratio")["missing_ratio"]

top10_missing_ratio_table = pd.DataFrame({
    'missing_ratio': top10_missing_ratio
})

top10_missing_ratio_table

In [ ]:
# check the contents of "OWN_CAR_AGE"

new_train_test.OWN_CAR_AGE.describe()

In [ ]:
new_train_test.OWN_CAR_AGE[new_train_test.OWN_CAR_AGE==0]

In [ ]:
print("ratio of 0 is {}%".format(len(new_train_test.OWN_CAR_AGE[new_train_test.OWN_CAR_AGE==0]) / len(new_train_test) * 100))

    - I guess that many people who did not own theirown car tended not to answer the question of the "OWN_CAR_AGE" column while some other people(only 0.6%) answered 0. This is because even 66% are missing values.

In [ ]:
# check the contents of "AMT_REQ_CREDIT_BUREAU_YEAR"

new_train_test.AMT_REQ_CREDIT_BUREAU_YEAR.describe()

    - This feature value means the number of enquiries to Credit Bureau about the client one day year (excluding last 3 months before application). Therefore, I guess missing values might equal to 0, since the number of enquiries shold be counted if people did so.
    
    - This prediction can apply to same kind of other 5 feature values such as "AMT_REQ_CREDIT_BUREAU_QRT" and "AMT_REQ_CREDIT_BUREAU_MON".

In [ ]:
# handle the missing values of the 7 columns by changing them to 0s

edit_missing_values = {"OWN_CAR_AGE":0,"AMT_REQ_CREDIT_BUREAU_YEAR":0,"AMT_REQ_CREDIT_BUREAU_QRT":0,"AMT_REQ_CREDIT_BUREAU_MON":0,
                       "AMT_REQ_CREDIT_BUREAU_DAY":0,"AMT_REQ_CREDIT_BUREAU_HOUR":0,"AMT_REQ_CREDIT_BUREAU_WEEK":0}

edit_train_test = new_train_test.fillna(value=edit_missing_values)

edit_train_test

In [ ]:
# compute AUC

compute_auc(edit_train_test)

    - The score is almost same with the previous one. (Exactly speaking, it slightly goes down.)

### 2nd Try by Focusing on the Feature Importance

In [ ]:
# check the contents of "ENTRANCES_MODE"

new_train_test.ENTRANCES_MODE.describe()

In [ ]:
# computer the missing ratio

print("missing ratio is {}%".format(new_train_test.ENTRANCES_MODE.isnull().sum() / len(new_train_test.ENTRANCES_MODE) * 100))

In [ ]:
# get top100 feature values strongly correlated with "ENTRANCES_MODE"

correlations_ENTRANCES_MODE = corr_mat.nlargest(101, "ENTRANCES_MODE")["ENTRANCES_MODE"].drop("ENTRANCES_MODE")

correlations_ENTRANCES_MODE_table = pd.DataFrame({
    'correlation': correlations_ENTRANCES_MODE
})

correlations_ENTRANCES_MODE_table

    - The correlations of "ENTRANCES_MODE" with the top2 feature values, "ENTRANCES_MEDI" and "ENTRANCES_AVG" are too high. So, I choose the third feature value, "BASEMENTAREA_MODE".

In [ ]:
# relationship between the missing values of "ENTRANCES_MODE" and "BASEMENTAREA_MODE"

indexer_1 = new_train_test['ENTRANCES_MODE'].isnull()
missing_1 = new_train_test['ENTRANCES_MODE'].copy()
missing_1[indexer_1] = -1
missing_1[~indexer_1] = 1
df_missing_1 = pd.DataFrame(missing_1)
df_missing_1.columns = ['MISSING']


df_ENTRANCES_MODE = new_train_test['ENTRANCES_MODE'].copy()
df_ENTRANCES_MODE[indexer_1] = 0.145471   # the mean of "ENTRANCES_MODE"
df_ENTRANCES_MODE = pd.DataFrame(df_ENTRANCES_MODE)
df_ENTRANCES_MODE.columns = ['ENTRANCES_MODE_MEANS']


df_ENTRANCES_MODE_missing = pd.concat([df_missing_1, df_ENTRANCES_MODE, new_train_test], axis=1)

ax = sns.lmplot(x='BASEMENTAREA_MODE', y='ENTRANCES_MODE_MEANS', fit_reg = False, data=df_ENTRANCES_MODE_missing, hue='MISSING', 
                palette='Greys_r', size=7, aspect=1.5)

    - There are no strong correlations.

In [ ]:
# handle the missing values of "ENTRANCES_MODE" by changing the missing values to the medians

edit_columns = []
for _ in train.iloc[:,48:94].columns:
    edit_columns.append(_)

edit_columns.remove("FONDKAPREMONT_MODE")
edit_columns.remove("HOUSETYPE_MODE")
edit_columns.remove('WALLSMATERIAL_MODE')
edit_columns.remove('EMERGENCYSTATE_MODE')

for _ in edit_columns:
    edit2_train_test = new_train_test.fillna(new_train_test[_].mean())

edit2_train_test

In [ ]:
# compute AUC

compute_auc(edit2_train_test)

    - The score goes down slightly again.

### 3rd Try

In [ ]:
# check the contents of "EXT_SOURCE_1"

new_train_test.EXT_SOURCE_1.describe()

In [ ]:
# get top 50 feature values strongly correlated with "EXT_SOURCE_1"

EXT_SOURCE_1_correlations = corr_mat.nlargest(51, "EXT_SOURCE_1")["EXT_SOURCE_1"].drop("EXT_SOURCE_1")

EXT_SOURCE_1_correlations_table = pd.DataFrame({
    'correlation': EXT_SOURCE_1_correlations
})

EXT_SOURCE_1_correlations_table

    - There are no plus storng correlations.

In [ ]:
# compare "EXT_SOURCE_1" with "TARGET"

new_train_test.EXT_SOURCE_1[new_train_test.TARGET==1].describe()

In [ ]:
new_train_test.EXT_SOURCE_1[new_train_test.TARGET==0].describe()

    - I need to take the same number of samples from "EXT_SOURCE_1" whose "TARGET" values is 0 with the number of "EXT_SOURCE_1" whose "TARGET" values is 1.

In [ ]:
# adjust the number of 2 kinds of samples

new_train_test.EXT_SOURCE_1[new_train_test.TARGET==0].sample(n=25000).describe()

    - The mean of the samples whose "TARGET" value is 1 smaller than the samples whose "TARGET" value is 0.

In [ ]:
# create a new feature value based on "EXT_SOURCE_1"

# add a spare column
new_train_test["ADD_EXT_SOURCE_1"] = float(0)

# substitute 1 to the samples whose mean is less than the mean of the 2 kinds of samples whose "TARGET" values is 0 or 1.
new_train_test.loc[new_train_test.EXT_SOURCE_1<(new_train_test.EXT_SOURCE_1[new_train_test.TARGET==1].mean()+ \
                                                new_train_test.EXT_SOURCE_1[new_train_test.TARGET==0].sample(n=25000).mean())/2, 
                   "ADD_EXT_SOURCE_1"] = 1

In [ ]:
# create a new feature value based on "EXT_SOURCE_2"

# add a spare column
new_train_test["ADD_EXT_SOURCE_2"] = float(0)

# substitute 1 to the samples whose mean is less than the mean of the 2 kinds of samples whose "TARGET" values is 0 or 1.
new_train_test.loc[new_train_test.EXT_SOURCE_2<(new_train_test.EXT_SOURCE_2[new_train_test.TARGET==1].mean()+ \
                                                new_train_test.EXT_SOURCE_2[new_train_test.TARGET==0].sample(n=25000).mean())/2, 
                   "ADD_EXT_SOURCE_2"] = 1

In [ ]:
# create a new feature value based on "EXT_SOURCE_3"

# add a spare column
new_train_test["ADD_EXT_SOURCE_3"] = float(0)

# substitute 1 to the samples whose mean is less than the mean of the 2 kinds of samples whose "TARGET" values is 0 or 1.
new_train_test.loc[new_train_test.EXT_SOURCE_3<(new_train_test.EXT_SOURCE_3[new_train_test.TARGET==1].mean()+ \
                                                new_train_test.EXT_SOURCE_3[new_train_test.TARGET==0].sample(n=25000).mean())/2, 
                   "ADD_EXT_SOURCE_3"] = 1

In [ ]:
# compute AUC

compute_auc(new_train_test)

    - The score goes down.

# Submittion Process From Here